In [1]:
# Import Deficiencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps

import requests
import json
import pprint

In [2]:
combined =pd.read_csv("combineddata.csv")
combined.head()

,Unnamed: 0,Store Number,Store Type,Address,City,State,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,0,4,Walmart Supercenter,2901 Highway 412 E,Siloam Springs,AR,72761,22314,31.5,48064,21146,3887,17.419557
1,1,8,Walmart Supercenter,1621 North Business 9,Morrilton,AR,72110,10913,40.0,37808,22314,2302,21.094108
2,2,5,Walmart Supercenter,1155 Hwy 65 North,Conway,AR,72032,32508,34.1,44686,22699,5311,16.337517
3,3,7,Walmart Supercenter,9053 Highway 107,Sherwood,AR,72120,33161,38.7,63575,30459,3654,11.018968
4,4,3,Walmart Supercenter,30983 Highway 441 S,Commerce,GA,30529,10766,36.8,44854,21262,1840,17.090842


In [3]:
citylatlong = pd.read_csv("citylatlong.csv")
# citylatlong.sort_values("City").head()
citylatlong.head()

,Unnamed: 0,City,State,City_State,Latitude,Longitude
0,0,Siloam Springs,AR,"Siloam Springs, AR",36.188136,-94.540496
1,1,Morrilton,AR,"Morrilton, AR",35.150917,-92.744054
2,2,Conway,AR,"Conway, AR",35.088696,-92.442101
3,3,Sherwood,AR,"Sherwood, AR",34.815091,-92.224315
4,4,Commerce,GA,"Commerce, GA",34.203997,-83.457106


In [4]:

city_state= pd.DataFrame({"City": citylatlong.groupby("City_State")["City"].first(), 
                        "State": citylatlong.groupby("City_State")["State"].first(),
                        "Latitude": citylatlong.groupby("City_State")["Latitude"].first(),
                        "Longitude": citylatlong.groupby("City_State")["Longitude"].first(),
                         "# of Stores": citylatlong.groupby("City_State")["City_State"].count()})
city_state.head(8)

,City,State,Latitude,Longitude,# of Stores
City_State,,,,,
"Abbeville, LA",Abbeville,LA,29.974650,-92.134292,1
"Aberdeen, MD",Aberdeen,MD,39.509556,-76.164120,1
"Aberdeen, NC",Aberdeen,NC,35.131547,-79.429479,1
"Aberdeen, SD",Aberdeen,SD,45.464698,-98.486483,1
"Aberdeen, WA",Aberdeen,WA,46.975371,-123.815722,1
"Abilene, TX",Abilene,TX,32.448736,-99.733144,4
"Abingdon, MD",Abingdon,MD,39.462236,-76.277222,1
"Abingdon, VA",Abingdon,VA,36.709834,-81.977348,1


In [5]:
walmart_df = pd.DataFrame(combined[["Address", "City", "State", "Zipcode", "Household Income"]])
walmart_df= walmart_df.assign(City_State=walmart_df.City.astype(str) +", " + walmart_df.State.astype(str))
walmart_df.head()

,Address,City,State,Zipcode,Household Income,City_State
0,2901 Highway 412 E,Siloam Springs,AR,72761,48064,"Siloam Springs, AR"
1,1621 North Business 9,Morrilton,AR,72110,37808,"Morrilton, AR"
2,1155 Hwy 65 North,Conway,AR,72032,44686,"Conway, AR"
3,9053 Highway 107,Sherwood,AR,72120,63575,"Sherwood, AR"
4,30983 Highway 441 S,Commerce,GA,30529,44854,"Commerce, GA"


In [6]:
wal_complete = pd.merge(walmart_df,
                         city_state[["Latitude", "Longitude", "# of Stores"]],
                          on= ["City_State", "City_State"],
                          how= "left")
wal_complete.head()

,Address,City,State,Zipcode,Household Income,City_State,Latitude,Longitude,# of Stores
0,2901 Highway 412 E,Siloam Springs,AR,72761,48064,"Siloam Springs, AR",36.188136,-94.540496,2
1,1621 North Business 9,Morrilton,AR,72110,37808,"Morrilton, AR",35.150917,-92.744054,1
2,1155 Hwy 65 North,Conway,AR,72032,44686,"Conway, AR",35.088696,-92.442101,3
3,9053 Highway 107,Sherwood,AR,72120,63575,"Sherwood, AR",34.815091,-92.224315,2
4,30983 Highway 441 S,Commerce,GA,30529,44854,"Commerce, GA",34.203997,-83.457106,1


In [7]:
# wal_complete.info()

In [8]:
lowIncome = wal_complete.loc[wal_complete["Household Income"] <= 35000]
midIncome = wal_complete.loc[(wal_complete["Household Income"] > 35000) & 
                             (wal_complete["Household Income"] <= 100000)]
highIncome = wal_complete.loc[wal_complete["Household Income"] > 100000]
lowIncome.head()

,Address,City,State,Zipcode,Household Income,City_State,Latitude,Longitude,# of Stores
10,1211 Highway 367 N,Newport,AR,72112,29180,"Newport, AR",35.604802,-91.281795,1
17,1201 N Service Rd E,Ruston,LA,71270,32729,"Ruston, LA",32.523205,-92.637927,2
24,1710 S 4Th St,Nashville,AR,71852,34665,"Nashville, AR",33.945669,-93.847129,1
36,1907 Se Washington St,Idabel,OK,74745,31394,"Idabel, OK",33.895665,-94.826328,1
42,1911 E Jackson St,Hugo,OK,74743,29665,"Hugo, OK",34.010660,-95.509682,1


In [9]:
inc = wal_complete["Household Income"].tolist()
incmin , incmax = min(inc), max(inc)
weightinc = [(val-incmin) / (incmax-incmin) for val in inc]

In [10]:
fig = gmaps.figure()
locations = lowIncome[["Latitude", "Longitude"]].astype(float)

lowincome_layer = gmaps.heatmap_layer(locations)

lowincome_layer.max_intensity= 10
lowincome_layer.weights=weightinc
# lowincome_layer.dissipating=False
# income_layer.point_radius = 1
lowincome_layer.gradient=["white", "purple"]
# lowincome_layer.opacity=0.5
fig.add_layer(lowincome_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
fig = gmaps.figure()
locations = midIncome[["Latitude", "Longitude"]].astype(float)

midincome_layer = gmaps.heatmap_layer(locations)

midincome_layer.max_intensity= 10
midincome_layer.weights=weightinc
# income_layer.dissipating=False
# income_layer.point_radius = 1
midincome_layer.gradient=["white", "teal"]
midincome_layer.opacity=0.5
fig.add_layer(midincome_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
fig = gmaps.figure()
locations = highIncome[["Latitude", "Longitude"]].astype(float)

highincome_layer = gmaps.heatmap_layer(locations)

highincome_layer.max_intensity= 10
highincome_layer.weights=weightinc
# highincome_layer.dissipating=False
# income_layer.point_radius = 1
highincome_layer.gradient=["white", "maroon"]
highincome_layer.opacity=0.5
fig.add_layer(highincome_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
fig = gmaps.figure()

fig.add_layer(lowincome_layer)
fig.add_layer(midincome_layer)
# fig.add_layer(highincome_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
fig = gmaps.figure()
locations = wal_complete[["Latitude", "Longitude"]].astype(float)

walmart_layer = gmaps.heatmap_layer(locations)

walmart_layer.max_intensity= 10
walmart_layer.weights=weightinc
# highincome_layer.dissipating=False
# income_layer.point_radius = 1

# walmart_layer.opacity=0.5
fig.add_layer(walmart_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Convert # of stores to list
stores = midIncome["# of Stores"].tolist()
cities = midIncome["City"].tolist()
state = midIncome["State"].tolist()
fig = gmaps.figure()

locations = midIncome[["Latitude", "Longitude"]].astype(float)
midwalmart_layer = gmaps.symbol_layer(
    locations, fill_color=['blue']*len(locations),
    stroke_color="blue", scale=2,
    info_box_content=[f"# of Walmart Stores in {city}, {state}: {store}" for city, state, store 
                      in zip(cities, state, stores)])

fig.add_layer(midwalmart_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Figure(layout=FigureLayout(height='420px'))